# 📸 OCR - 이미지에서 텍스트 추출

이 노트북은 beanllm의 OCR (Optical Character Recognition) 기능을 학습합니다. 이미지에서 텍스트를 추출하고 문서를 디지털화할 수 있습니다.

## 📋 목차

- [1. OCR 기본 사용](#1-ocr-기본-사용)
- [2. 다양한 OCR 엔진](#2-다양한-ocr-엔진)
- [3. 이미지 전처리](#3-이미지-전처리)
- [4. LLM 후처리](#4-llm-후처리)
- [5. 배치 처리](#5-배치-처리)
- [6. PDF OCR](#6-pdf-ocr)
- [7. 실전 예제](#7-실전-예제)

---

## 🎯 학습 목표

이 노트북을 완료하면:
- ✅ beanOCR을 사용하여 이미지에서 텍스트를 추출할 수 있습니다
- ✅ 다양한 OCR 엔진을 선택할 수 있습니다
- ✅ 이미지 전처리를 적용할 수 있습니다
- ✅ LLM 후처리로 정확도를 향상시킬 수 있습니다
- ✅ 배치 처리와 PDF OCR을 수행할 수 있습니다

---

## 📚 사전 요구사항

- [01_setup_and_installation.ipynb](01_setup_and_installation.ipynb) 완료
- [02_core_client.ipynb](02_core_client.ipynb) 완료


## 1. OCR 기본 사용

beanOCR은 7개의 OCR 엔진을 통합하여 사용하기 쉬운 인터페이스를 제공합니다.

> 💡 **OCR의 핵심**:
> - 이미지에서 텍스트 추출
> - 여러 OCR 엔진 지원 (PaddleOCR, EasyOCR, TrOCR, Nougat, Surya, Tesseract, Cloud)
> - 이미지 전처리로 품질 향상
> - LLM 후처리로 98%+ 정확도 달성


In [ ]:
from beanllm.domain.ocr import beanOCR

# 기본 OCR 사용
ocr = beanOCR(engine="paddleocr", language="ko")

# 이미지에서 텍스트 추출
# result = ocr.recognize("scanned_image.jpg")

print("✅ beanOCR 생성 완료!")
print(f"📊 엔진: {ocr.config.engine}")
print(f"📊 언어: {ocr.config.language}")

# 결과 예시
print("\n💡 OCR 결과 구조:")
print("  - result.text: 추출된 텍스트")
print("  - result.confidence: 신뢰도 (0.0-1.0)")
print("  - result.lines: 텍스트 라인 리스트")
print("  - result.bboxes: 바운딩 박스 리스트")
print("  - result.processing_time: 처리 시간")

# 실제 사용 예제
# print(f"\n📝 추출된 텍스트: {result.text}")
# print(f"📊 신뢰도: {result.confidence:.2%}")
# print(f"📊 처리 시간: {result.processing_time:.2f}초")

## 2. 다양한 OCR 엔진

beanOCR은 여러 OCR 엔진을 지원합니다. 각 엔진은 특정 용도에 최적화되어 있습니다.

### 2.1 엔진 비교


In [ ]:
# 지원하는 OCR 엔진
engines = {
    "paddleocr": "PaddleOCR - 다국어, 빠름, 정확도 높음 (권장)",
    "easyocr": "EasyOCR - 다국어, 설치 간편",
    "tesseract": "Tesseract - 오픈소스, 안정적",
    "trocr": "TrOCR - Transformer 기반, 고품질",
    "nougat": "Nougat - 학술 논문 특화",
    "surya": "Surya - 다국어, 레이아웃 분석",
    "cloud": "Cloud API - Google/Amazon OCR",
}

print("🔧 지원하는 OCR 엔진:")
for engine, description in engines.items():
    print(f"  - {engine}: {description}")

print("\n💡 엔진 선택 가이드:")
print("  - 일반 문서: paddleocr (권장)")
print("  - 학술 논문: nougat")
print("  - 다국어 문서: surya")
print("  - 빠른 처리: easyocr")
print("  - 최고 정확도: trocr + LLM 후처리")

### 2.2 엔진 전환

같은 이미지에 대해 다른 엔진을 시도할 수 있습니다.


In [ ]:
async def compare_engines():
    """엔진 비교 예제"""

    image_path = "sample_image.jpg"

    engines_to_test = ["paddleocr", "easyocr", "tesseract"]

    print("🔍 OCR 엔진 비교:\n")
    for engine in engines_to_test:
        try:
            ocr = beanOCR(engine=engine, language="ko")
            # result = ocr.recognize(image_path)

            print(f"✅ {engine}:")
            # print(f"   텍스트: {result.text[:50]}...")
            # print(f"   신뢰도: {result.confidence:.2%}")
            # print(f"   처리 시간: {result.processing_time:.2f}초\n")
            print(f"   (실제 실행 시 결과 표시)")
        except Exception as e:
            print(f"❌ {engine}: 오류 - {e}\n")


# 실행 (주석 해제)
# await compare_engines()
print("💡 위의 주석을 해제하여 실행하세요.")

## 3. 이미지 전처리

이미지 전처리를 통해 OCR 정확도를 향상시킬 수 있습니다.

### 3.1 전처리 옵션


In [ ]:
from beanllm.domain.ocr import OCRConfig

# 전처리 옵션이 포함된 OCR 설정
config = OCRConfig(
    engine="paddleocr",
    language="ko",
    # 전처리 옵션
    preprocess=True,  # 전처리 활성화
    denoise=True,  # 노이즈 제거
    deskew=True,  # 기울기 보정
    binarize=True,  # 이진화
    threshold=127,  # 이진화 임계값
)

ocr = beanOCR(config=config)

print("✅ 전처리 옵션이 포함된 OCR 생성!")
print("📊 전처리 기능:")
print("  - 노이즈 제거: 노이즈 감소")
print("  - 기울기 보정: 문서 기울기 자동 보정")
print("  - 이진화: 흑백 변환으로 명확도 향상")
print("  - 크기 조정: 최적 해상도로 조정")

# 사용 예제
# result = ocr.recognize("noisy_image.jpg")
# print(f"\n📝 전처리 후 텍스트: {result.text[:100]}...")

## 4. LLM 후처리

LLM을 사용하여 OCR 결과를 정제하면 98%+ 정확도를 달성할 수 있습니다.

### 4.1 LLM 후처리 활성화


In [ ]:
# LLM 후처리가 활성화된 OCR
ocr = beanOCR(
    engine="paddleocr",
    language="ko",
    enable_llm_postprocessing=True,  # LLM 후처리 활성화
    llm_model="gpt-4o-mini",  # LLM 모델
)

print("✅ LLM 후처리 활성화!")
print("📊 LLM 후처리 기능:")
print("  - 오타 수정: OCR 오류 자동 수정")
print("  - 문맥 복원: 문맥에 맞게 텍스트 보정")
print("  - 형식 정리: 일관된 형식으로 정리")
print("  - 정확도 향상: 98%+ 정확도 달성")

# 사용 예제
# result = ocr.recognize("noisy_image.jpg")
# print(f"\n📝 LLM 후처리 후 텍스트: {result.text[:200]}...")
# print(f"📊 신뢰도: {result.confidence:.2%}")

## 5. 배치 처리

여러 이미지를 한 번에 처리할 수 있습니다.


In [ ]:
# 배치 처리
ocr = beanOCR(engine="paddleocr", language="ko")

# 여러 이미지 처리
image_paths = ["image1.jpg", "image2.jpg", "image3.jpg"]
# results = ocr.batch_recognize(image_paths)

print("✅ 배치 처리 준비 완료!")
print(f"📊 처리할 이미지 수: {len(image_paths)}개")

# 결과 처리 예제
# for i, result in enumerate(results, 1):
#     print(f"\n이미지 {i}:")
#     print(f"  텍스트: {result.text[:100]}...")
#     print(f"  신뢰도: {result.confidence:.2%}")

print("\n💡 배치 처리 특징:")
print("  - 병렬 처리로 빠른 속도")
print("  - 자동 재시도 (실패 시)")
print("  - 진행 상황 추적")

## 6. PDF OCR

PDF 파일의 특정 페이지에서 텍스트를 추출할 수 있습니다.


In [ ]:
# PDF OCR
ocr = beanOCR(engine="paddleocr", language="ko")

# PDF 페이지 OCR
# result = ocr.recognize_pdf_page("document.pdf", page_num=0)

print("✅ PDF OCR 준비 완료!")
print("📊 PDF OCR 기능:")
print("  - 특정 페이지 선택")
print("  - 전체 페이지 처리")
print("  - 이미지 페이지 자동 감지")

# 사용 예제
# print(f"📄 페이지 0 텍스트: {result.text[:200]}...")
# print(f"📊 신뢰도: {result.confidence:.2%}")

# 전체 PDF 처리
# all_results = ocr.recognize_pdf("document.pdf")
# print(f"\n📚 전체 페이지 수: {len(all_results)}")

## 7. 실전 예제

### 7.1 문서 디지털화 시스템

스캔된 문서를 디지털 텍스트로 변환하는 시스템입니다.


In [ ]:
async def document_digitization():
    """문서 디지털화 시스템"""

    # 고정확도 OCR 설정
    ocr = beanOCR(
        engine="paddleocr",
        language="ko",
        enable_llm_postprocessing=True,
        llm_model="gpt-4o-mini",
        preprocess=True,
    )

    # 스캔된 문서 처리
    scanned_docs = ["doc1.jpg", "doc2.jpg", "doc3.jpg"]

    print("📄 문서 디지털화 시작...\n")

    # 배치 처리
    # results = ocr.batch_recognize(scanned_docs)

    # 결과 저장
    # for i, result in enumerate(results, 1):
    #     output_file = f"digitized_doc_{i}.txt"
    #     with open(output_file, "w", encoding="utf-8") as f:
    #         f.write(result.text)
    #     print(f"✅ 문서 {i} 디지털화 완료: {output_file}")

    print("💡 문서 디지털화 시스템:")
    print("  - 스캔된 이미지 → 텍스트 파일")
    print("  - LLM 후처리로 정확도 향상")
    print("  - 배치 처리로 효율성 향상")


# 실행 (주석 해제)
# await document_digitization()
print("💡 위의 주석을 해제하여 실행하세요.")

### 7.2 바운딩 박스 시각화

OCR 결과의 바운딩 박스를 시각화할 수 있습니다.


In [ ]:
from PIL import Image, ImageDraw, ImageFont


def visualize_bboxes(image_path, ocr_result):
    """바운딩 박스 시각화"""

    # 이미지 로드
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)

    # 각 텍스트 라인의 바운딩 박스 그리기
    for line in ocr_result.lines:
        bbox = line.bbox
        # 바운딩 박스 그리기
        draw.rectangle([(bbox.x1, bbox.y1), (bbox.x2, bbox.y2)], outline="red", width=2)
        # 텍스트 표시
        draw.text((bbox.x1, bbox.y1 - 20), line.text[:20], fill="red")

    # 결과 저장
    output_path = "annotated_image.jpg"
    img.save(output_path)
    print(f"✅ 시각화 완료: {output_path}")


# 사용 예제
# ocr = beanOCR(engine="paddleocr")
# result = ocr.recognize("image.jpg")
# visualize_bboxes("image.jpg", result)

print("💡 바운딩 박스 시각화:")
print("  - 각 텍스트 라인의 위치 표시")
print("  - 신뢰도에 따른 색상 구분 가능")
print("  - OCR 결과 검증에 유용")

## 📸 시각화 예제

### OCR 파이프라인 시각화

> 💡 **이미지 필요**: OCR 파이프라인의 단계를 보여주는 다이어그램

```python
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

fig, ax = plt.subplots(figsize=(12, 6))

# OCR 파이프라인 단계
steps = ["원본 이미지", "전처리", "OCR 엔진", "LLM 후처리", "최종 텍스트"]

x_positions = [0, 2.5, 5, 7.5, 10]

for i, (step, x) in enumerate(zip(steps, x_positions)):
    color = "lightblue" if i == 0 or i == len(steps)-1 else "lightgreen"
    rect = mpatches.Rectangle((x-1, -0.3), 2, 0.6, facecolor=color, edgecolor='black')
    ax.add_patch(rect)
    ax.text(x, 0, step, ha='center', va='center', fontsize=10)
    
    if i < len(steps) - 1:
        ax.arrow(x+1, 0, 0.5, 0, head_width=0.1, head_length=0.1, fc='black', ec='black')

ax.set_xlim(-1.5, 11.5)
ax.set_ylim(-1, 1)
ax.axis('off')
plt.title('OCR 파이프라인', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()
```

**필요한 이미지**:
- 플로우차트: OCR 파이프라인의 5단계
- 원본 이미지 → 전처리 → OCR 엔진 → LLM 후처리 → 최종 텍스트
- 각 단계는 박스로 표시
- 화살표로 데이터 흐름 표시
- 바운딩 박스가 그려진 이미지 예시 (선택적)


## 8. 다음 단계

OCR을 마스터했습니다! 다음 튜토리얼로 진행하세요:

### 📚 추천 학습 순서

1. **[10_audio_processing.ipynb](10_audio_processing.ipynb)** - Audio Processing
   - 음성 인식 및 합성
   - 오디오 처리

2. **[11_web_search.ipynb](11_web_search.ipynb)** - Web Search
   - 웹 검색 통합
   - 실시간 정보 활용

3. **[14_production_features.ipynb](14_production_features.ipynb)** - Production Features
   - 프로덕션 환경 최적화
   - 모니터링 및 로깅

### 🔗 관련 문서

- [API Reference](../API_REFERENCE.md#ocr) - OCR API 상세 문서
- [README.md](../../README.md) - 프로젝트 개요

---

**✅ OCR 완료! 다음 노트북으로 진행하세요.**
